In [ ]:
import pandas as pd
import numpy as np
import re
import persian

In [ ]:
df = pd.read_excel(r'pishnahdkamel.XLS',usecols="G")

In [ ]:
df = df[pd.notnull(df['sharhpishnahad'])]
df = df[df.sharhpishnahad.duplicated()==False]
df.reset_index(inplace = True)
del df['index']

In [ ]:
stopwords = []
file = open('stopwords.txt',encoding = 'utf-8').read()
[stopwords.append(x) for x in file.split()]
stopwords = set(stopwords)

In [ ]:
def removing_stopwords(text):
    text =str(text)
    filtered_tokens = [token for token in text.split() if token not in stopwords]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

In [ ]:
def convert_char(text):
    text =str(text)
    s = ''
    for word in text:
        s = s + persian.convert_ar_characters(word)
    return s

In [ ]:
def sent_to_word(data):
    data = str(data)
    words = []
    for x in data.split():
        words.append(x)
    return words

In [ ]:
def sent_vectorizer(sent):
    sent_vec =[]
    numw = 0
    for w in sent:
        try:
            if numw == 0:
                sent_vec = ft.get_word_vector(w)
            else:
                sent_vec = np.add(sent_vec, ft.get_word_vector(w))
            numw += 1
        except:
            pass
    return np.asarray(sent_vec) / numw

In [ ]:
sent_words = []
for i in df.sharhpishnahad.index:
    temp = removing_stopwords(df.sharhpishnahad[i])
    temp = convert_char(temp)
    temp = removing_stopwords(temp)
    temp = sent_to_word(temp)
    sent_words.append(temp)

In [ ]:
with open("input_file.txt", "w",encoding = 'utf-8') as txt_file:
    for line in sent_words:
        txt_file.write(" ".join(line) + "\n")

In [ ]:
ft = fasttext.train_unsupervised('input_file.txt', model='skipgram', dim = 300)

In [ ]:
vec = []
for sentence in sent_words:
    vec.append(sent_vectorizer(sentence))

In [ ]:
del_row_id = []
del_row_value = []
for i in range(len(vec)):
    if len(vec[i]) == 0:
        del_row_id.append(i)
        tt = vec[i]
        del_row_value.append(tt)

In [ ]:
df = df.drop(df.index [del_row_id])

In [ ]:
df.reset_index(drop = True, inplace = True)

In [ ]:
new_vec = []
for e in vec:
    if e not in del_row_value:
        new_vec.append(e)

In [ ]:
vec_array = np.vstack(new_vec)

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
import matplotlib.pyplot as plt
sse = {}
for k in range(2, 10):
    kmeans_s = KMeans(n_clusters=k, max_iter=100).fit(vec_array)
    sse[k] = kmeans_s.inertia_
plt.figure()
plt.plot(list(sse.keys()), list(sse.values()))
plt.xlabel("Number of cluster")
plt.ylabel("SSE")
plt.show()

In [ ]:
kmeans = KMeans(n_clusters = 3, n_init = 50, random_state=1)
kmeans.fit(vec_array)

In [ ]:
labels = kmeans.labels_

In [ ]:
df.loc[:,'labels'] = kmeans.labels_

In [ ]:
df.labels.value_counts()

In [ ]:
from sklearn import metrics
metrics.silhouette_score(vec_array, labels, metric='euclidean')

In [ ]:
df.to_excel('Barez_2_without_AE.xlsx', index=False)